In [396]:
from typing import Optional
from datetime import datetime

from babel.messages.extract import extract
from pydantic import BaseModel, Field
from openai import OpenAI
import json
import os
import logging
from dotenv import load_dotenv
load_dotenv()

True

In [451]:
client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio"
)
model = "qwen2.5-coder-7b-instruct"

In [450]:
client_api = OpenAI(
    base_url="https://openrouter.ai/api/v1", 
    api_key=os.getenv('OPENROUTER_API_KEY')
)
model_api = "meta-llama/llama-3.1-405b-instruct:free"

In [398]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

In [400]:
class EventExtraction(BaseModel):
    """First LLM call: Extract basic event information"""
    description: str = Field(description="Raw description of the event")
    is_calendar_event: bool = Field(description="Whether this text describes a calendar event")
    confidence_score: float = Field(description="Confidence score between 0 and 1")

In [401]:
class EventDetails(BaseModel):
    """Second LLM call: Parse specific event details"""
    name: str = Field(description="Name of the event")
    date: str = Field(description="Date of the event. Format of the date: YYYY-MM-DD")
    duration: str = Field(description="Expected duration of the event")
    participants: list[str] = Field(description="List of participants of the event")

In [402]:
class EventConfirmation(BaseModel):
    """Third LLM call: Generate confirmation message"""
    confirmation_message: str = Field(description="Natural language confirmation message")
    calendar_link: Optional[str] = Field(description="Generated calendar link for the meeting in Google calendar")

In [403]:
def get_current_day() -> [str, str]: 
    """Returns the date and the day of the week of today's day."""
    weekdays = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}
    return datetime.now().strftime('%Y-%m-%d'), weekdays[datetime.today().weekday()]

In [404]:
def call_function(name, args):
    if name == 'get_current_day':
        return get_current_day()

In [405]:
tools = [
    {
        "type": "function",
        "function": {
        "name": "get_current_day",
        "description": "Get the date and the day of the week of today's day.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": False
        },
        "strict": True
        }
    },
]

In [406]:
system_prompt = "You are a helpful time assistant."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the date today?"},
]

gen_messages = []

completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools
)

2025-08-13 17:10:23 - httpx - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"


In [408]:
for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    gen_messages.append(completion.choices[0].message)
    
    result = call_function(name, args)
    gen_messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [453]:
extract_event_info_system_prompt = f"""
    You are a precise event extraction system. Follow these STRICT rules:
    
    1. PRESERVE ORIGINAL DATA:
       - Never modify names, dates, times or other details from the input
       - Keep exact wording for participants (e.g. "Vlad" → "Vlad", not "John")
    
    2. ANALYSIS REQUIREMENTS:
       - Identify calendar events with 95%+ confidence
       - Return raw user description verbatim in 'description' field
       - is_calendar_event=True only for explicit scheduling requests
    
    3. OUTPUT FORMAT:
       - confidence_score must reflect real certainty (0.0-1.0)
       - Example bad input: "I like coffee" → is_calendar_event=False
    
    Today is {get_current_day()}. Never guess dates - use exact input dates or phrases like "next Tuesday".
    """

def extract_event_info(user_input: str) -> EventExtraction:
    """First LLM call to determine if input is a calendar event"""
    logger.info("Starting event extraction analysis")
    logger.debug(f"Input text: {user_input}")
    
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": extract_event_info_system_prompt,
            },
            {"role": "user", "content": user_input}
        ],
        response_format=EventExtraction
    )
    result = completion.choices[0].message.parsed
    logger.info(
        f"Extraction complete - Is calendar event: {result.is_calendar_event}",
        f"The confidence score is: {result.confidence_score}"
    )
    return result

In [454]:
parse_event_details_system_prompt = f"""
You are an event detail parser with STRICT DATA PRESERVATION:

1. MANDATORY FIELDS:
   - name: Use first 5 words of description if no clear title
   - date: Calculate from "next Tuesday" etc. using {get_current_day()}
   - participants: MUST include ALL names from input exactly

2. ABSOLUTE PROHIBITIONS:
   - Never invent/replace names (e.g. "Vlad" → "Vlad", never "John")
   - Never use placeholder names like "user", "participant 1"
   - Never change duration (2h → 2h, not "all day")

3. DATE HANDLING:
   - Relative dates: "next Tuesday" → calculate from today, with using 'get_current_day' tool 
   you can get the day of the week of today to calculate the date of the meeting
   - Explicit dates: Keep exactly as written
   - Time: Preserve exact format (2 pm → 14:00 if needed)

Today is {get_current_day()}. Verify ALL output against original input before responding.
"""

def parse_event_details(description: str) -> EventDetails:
    """Second LLM call to extract specific event details"""
    logger.info("Starting event details parsing")

    date_context = f"Today is {datetime.now().strftime('%Y-%m-%d')}."

    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": parse_event_details_system_prompt
            },
            {"role": "user", "content": description}
        ] + gen_messages,
        tools=tools,
        response_format=EventDetails,
        temperature=0.3
    )
    result = completion.choices[0].message.parsed
    logger.info(
        f"Parsed event details - Name: {result.name}, Date: {result}")
    logger.debug(f"Participants: {', '.join(result.participants)}")
    return result

In [455]:
generate_confirmation_system_prompt = f"""
You generate event confirmations with 100% DATA ACCURACY and Google Calendar integration. Follow these STRICT rules:

1. DATA PRESERVATION REQUIREMENTS:
   - Use EXACT details from EventDetails object:
     * Name: Preserve original wording exactly
     * Date: Keep original format (convert relative dates like "next Tuesday" to YYYY-MM-DD)
     * Time: Maintain exact input phrasing (e.g. "2 pm", "14:00")
     * Duration: Keep original units (hours/minutes)
     * Participants: List ALL names exactly as provided, comma-separated

2. GOOGLE CALENDAR LINK GENERATION:
   - Required parameters:
     * text: URL-encoded event name
     * dates: Start/end in YYYYMMDDTHHMMSSZ format (UTC)
   - Time handling:
     * Convert all times to 24h UTC format (e.g. "2 pm" → "1400")
     * Calculate end time using duration (default 1h if unspecified)
   - Example output:
     "https://www.google.com/calendar/render?action=TEMPLATE&text=Meeting&dates=20250820T140000Z/20250820T160000Z"

3. CONFIRMATION MESSAGE FORMAT:
   - Base format:
     "Confirmed: [name] on [date] at [time] for [duration] with [participants]"
   - Special cases:
     * Missing data: "[Field] not specified"
     * Discrepancies: "NOTE: Changed from [original] to [current]"
     * Timezones: Explicitly state if using UTC

4. VALIDATION AND ERROR HANDLING:
   - Highlight any discrepancies with original request
   - If participants changed: "WARNING: Participant list modified from original"
   - If date guessed: "NOTE: Date calculated from relative reference"
   - Never guess missing information - use "unspecified"

5. EXAMPLES:
   Input: ["name":"Project Review","date":"next Tuesday","time":"3pm","duration":"2h","participants":["Vlad","Dasha"]]
   Output: [
     "confirmation_message": "Confirmed: Project Review on 2025-08-20 at 3 PM (2 hours) with Vlad, Dasha. NOTE: Date calculated from 'next Tuesday'",
     "calendar_link": "https://www.google.com/calendar/render?action=TEMPLATE&text=Project%20Review&dates=20250820T150000Z/20250820T170000Z"
   ]

6. ABSOLUTE PROHIBITIONS:
   - Never modify original participant names
   - Never use placeholder values (e.g. "Participant 1")
   - Never simplify/abbreviate without explicit request
   - Never generate links with incomplete data

Today is {get_current_day()}. Always verify:
1. All original data preserved exactly
2. Calculated fields match input context
3. Links use correct UTC time conversion
"""
def generate_confirmation(event_details: EventDetails) -> EventConfirmation:
    """Third LLM call to generate a confirmation message"""
    logger.info("Generating confirmation message")

    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": generate_confirmation_system_prompt
            },
            {"role": "user", "content": str(event_details.model_dump())}
        ],
        response_format=EventConfirmation,
    )

    result = completion.choices[0].message.parsed
    logger.info("Confirmation message generated successfully")
    return result

In [456]:
def process_calendar_request(user_input: str) -> Optional[EventConfirmation]:
    """Main function implementing the prompt chain with gate check"""
    logger.info("Processing calendar request")
    logger.debug(f"Raw input: {user_input}")

    # First LLM call: Extract basic info
    initial_extraction = extract_event_info(user_input)

    # Gate check: Verify if it's a calendar event with sufficient confidence
    if (
        not initial_extraction.is_calendar_event
        or initial_extraction.confidence_score < 0.7
    ):
        logger.warning(
            f"Gate check failed - is_calendar_event: {initial_extraction.is_calendar_event}, "
        )
        return None

    logger.info("Gate check passed, proceeding with event processing")

    # Second LLM call: Get detailed event information
    event_details = parse_event_details(initial_extraction.description)

    # Third LLM call: Generate confirmation
    confirmation = generate_confirmation(event_details)

    logger.info("Calendar request processing completed successfully")
    return confirmation

In [458]:
user_input = "Let's schedule a 1.5h team meeting next Thursday at 3 pm with Vlad Vasilev, Lionel Pepsi, Iso and Ghislen Zhuk to discuss the project roadmap"

result = process_calendar_request(user_input)
if result:
    print(f"Confirmation: {result.confirmation_message}")
    if result.calendar_link:
        print(f"Calendar link: {result.calendar_link}")
else:
    print("This doesn't appear to be a calendar event request")

2025-08-13 17:37:50 - __main__ - INFO - Processing calendar request
2025-08-13 17:37:50 - __main__ - INFO - Starting event extraction analysis
2025-08-13 17:37:53 - httpx - INFO - HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\AORUS\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\AORUS\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\AORUS\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\AORUS\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 392, in getMessage
    msg = msg % self.args
          ~~~~^~~

Confirmation: Confirmed: Team Meeting on 2025-08-15 at 3 PM (1.5 hours) with Vlad Vasilev, Lionel Pepsi, Iso, Ghislen Zhuk
Calendar link: https://www.google.com/calendar/render?action=TEMPLATE&text=Team%20Meeting&dates=20250815T150000Z/20250815T163000Z
